In [1]:
#RUN AN EXAMPLE scMRA TO GENERATE RLOC FOR A NETWORK WITH EXACTLY 13 EDGES WITH 500 CELLS AND 20% NOISE
#(for MRA figure)

import sys
sys.path.insert(0, '../../helperScripts/')
from helperFunctions import *
import scmra
import os
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math
import pandas as pd
from datetime import datetime
import pickle
import seaborn as sns; sns.set_theme()
from matplotlib.pyplot import figure
import scmra
import sys

CELLNUM = 1000
NOISE = 0.2

In [18]:
#just to test run-time
rglob, rtot, pDict = prepare_orton_data(wt, 500, 0.0)

In [19]:
#44 sec for 1000 cells
#3 sec for 200 cells
#18 sec for 500 cells

#500cells with 0,8 noise is 28sec/ but 0 noise is only 12 sec

scd = scmra.ScData(rglob=rglob, rtot = rtot)
#make MRA simulation
p = scmra.ScMraProblem(scd, eta=0.02)
p.cpx.solve()
s = scmra.ScMraResult(p)

print(s.rloc)

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Tried aggregator 2 times.
MIQP Presolve eliminated 0 rows and 1 columns.
Aggregator did 110 substitutions.
Reduced MIQP has 5610 rows, 5951 columns, and 66330 nonzeros.
Reduced MIQP has 330 binaries, 0 generals, 0 SOSs, and 330 indicators.
Reduced MIQP objective Q matrix has 5500 nonzeros.
Presolve time = 0.09 sec. (30.01 ticks)
Probing time = 0.00 sec. (1.66 ticks)
Tried aggregator 1 time.
Reduced MIQP has 5610 rows, 5951 columns, and 66330 nonzeros.
Reduced MIQP has 330 binaries, 0 generals, 0 SOSs, and 330 indicators.
Reduced MIQP objective Q matrix has 5500 nonzeros.
Presolve time = 0.03 sec. (18.55 ticks)
Classifier predicts products in MIQP should be linearized.
Probing time = 0.00 sec. (1.66 ticks)
Clique table members: 110.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 32 threads.
Root relaxatio

In [2]:


rglob, rtot, pDict = prepare_orton_data(wt, CELLNUM, NOISE)

#run an eta-approximation, returns quickly an eta which approximatews a network with 13 edges
eta, scMraResult, edgeNumber = get_suitable_solution_fast(rglob, rtot, etaGuesses=30)

#thew true network has 13 edges
print(edgeNumber)

#add simulation output to result
scMraResult.rloc.to_csv("./bin/rloc_#1000_N20.tsv", sep = "\t")

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Tried aggregator 2 times.
MIQP Presolve eliminated 0 rows and 1 columns.
Aggregator did 110 substitutions.
Reduced MIQP has 11110 rows, 11451 columns, and 132330 nonzeros.
Reduced MIQP has 330 binaries, 0 generals, 0 SOSs, and 330 indicators.
Reduced MIQP objective Q matrix has 11000 nonzeros.
Presolve time = 0.45 sec. (91.04 ticks)
Probing time = 2.31 sec. (1.40 ticks)
Tried aggregator 1 time.
Reduced MIQP has 11110 rows, 11451 columns, and 132330 nonzeros.
Reduced MIQP has 330 binaries, 0 generals, 0 SOSs, and 330 indicators.
Reduced MIQP objective Q matrix has 11000 nonzeros.
Presolve time = 0.22 sec. (52.53 ticks)
Classifier predicts products in MIQP should be linearized.
Probing time = 2.75 sec. (1.40 ticks)
Clique table members: 110.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 32 threads.
Root r

In [3]:
print(eta)

0.05


In [2]:
#generate several networks with different etas, to plot networks form those rlocs
# => you can see how eta efects FP and TP edges in the network

#add header to result
results = []
edges = []
errorList = []

noise = 0.2
cell = 1000
etas = [0.0001, 0.0003, 0.0006, 0.001, 0.002, 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.65, 0.7, 0.735, 0.77, 0.8]
        #if we want to estimate eta/ however it looks like it does not 'guess' the node number very well
        #better run a bunch of eta guesses
        #etaList = estimate_eta_list(cell, noise)
rglob, rtot, pDict = prepare_orton_data(wt, cell, noise)

for eta in (etas):

    #manually run with a couple of etas

    scd = scmra.ScData(rglob=rglob, rtot = rtot)

    #make MRA simulation
    p = scmra.ScMraProblem(scd, eta=eta)
    p.cpx.solve()
    s = scmra.ScMraResult(p)

    results.append(s)

    numEdges = s.imap.sum().sum()
    edges.append(numEdges)

    n_res = np.size(s.residuals_complete) + \
            np.size(s.residuals_incomplete)
    error = 0
    error += np.sum(np.array(np.square(s.residuals_complete)))/n_res
    error += np.sum(np.array(np.square(s.residuals_incomplete)))/n_res

    errorList.append(error)

data = {
  "etas": etas,
  "edges": edges,
  "error": errorList
}
resultFrame = pd.DataFrame(data)
resultFrame.to_csv("./bin/exampleRlocetaErrorFrame.csv")

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Tried aggregator 2 times.
MIQP Presolve eliminated 0 rows and 1 columns.
Aggregator did 110 substitutions.
Reduced MIQP has 11110 rows, 11451 columns, and 132330 nonzeros.
Reduced MIQP has 330 binaries, 0 generals, 0 SOSs, and 330 indicators.
Reduced MIQP objective Q matrix has 11000 nonzeros.
Presolve time = 0.27 sec. (91.04 ticks)
Probing time = 0.18 sec. (1.40 ticks)
Tried aggregator 1 time.
Reduced MIQP has 11110 rows, 11451 columns, and 132330 nonzeros.
Reduced MIQP has 330 binaries, 0 generals, 0 SOSs, and 330 indicators.
Reduced MIQP objective Q matrix has 11000 nonzeros.
Presolve time = 0.13 sec. (52.53 ticks)
Classifier predicts products in MIQP should be linearized.
Probing time = 0.26 sec. (1.40 ticks)
Clique table members: 110.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 32 threads.
Root r

In [32]:
results[0].rloc.to_csv("/DATA/t.stohn/MRA/scMRA-analysis/bin/rloc_0.0001.tsv", sep = "\t")
results[4].rloc.to_csv("/DATA/t.stohn/MRA/scMRA-analysis/bin/rloc_0.1.tsv", sep = "\t")
results[9].rloc.to_csv("/DATA/t.stohn/MRA/scMRA-analysis/bin/rloc_0.5.tsv", sep = "\t")